In [6]:
from  test_model import do_test
from sequence_model import CTPAModel  
from sequence_dataset import ChunkDataset
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd

In [7]:
import pickle
import pandas as pd
#reading embeddings with and without offset
file_paths0 =  "C:\\Users\\preet\\Documents\\penet\\results\\best_result_shift_0\\embs_test.pickle"

file_paths1 = "C:\\Users\\preet\\Documents\\penet\\results\\best_result_shift_1\\embs_test.pickle"

def return_loaded_files(file_path):
  
  
    with open(file_path, 'rb') as f:
            # Load the pickled object from the file
            loaded_data_test = pickle.load(f)
    return loaded_data_test


In [8]:
emb_test = return_loaded_files(file_paths0)
emb_test1 = return_loaded_files(file_paths1)
df_test = pd.DataFrame(emb_test)
df_test["emb1"] = emb_test1["emb"]
df_test["prob1"] = emb_test1["prob"]
test_dataset = ChunkDataset(df_test)
test_dataloader = DataLoader(test_dataset, batch_size=2, num_workers=0,drop_last=True)


In [9]:
embed_dim = 384
num_heads = 8
num_layers = 4

model = CTPAModel(embed_dim, num_heads, num_layers, drop_prob=0.1)
model.to("cuda")

CTPAModel(
  (pos_embedding): Embedding(1024, 384)
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
    )
    (linear1): Linear(in_features=384, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=384, bias=True)
    (norm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
        )
        (linear1): Linear(in_features=384, out_features=2048, bias=True)
        (dropout): Dropout(p

In [11]:
state_dict = torch.load(r"C:\Users\preet\Documents\penet\results_sequence\best_model.pth") 
model.load_state_dict(state_dict)
model.eval()
metrics, output = do_test(test_dataloader, model)
del metrics["Loss"]
print(metrics)
metrics_df = pd.DataFrame.from_dict([metrics])
print(metrics_df.head())

  0%|                                                                                           | 0/81 [00:00<?, ?it/s]C:\Users\preet\Documents\penet\sequence_model\test_model.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output.extend(F.softmax(outputs).cpu().numpy()[:, 1])
100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [00:01<00:00, 77.29it/s]

Test loss0.658, F1 0.706, Acc 0.759, BA 0.840, precision 0.415, recall 0.964
cm[[96 38]
 [ 1 27]]
{'Accuracy': 0.7592592592592593, 'Sensitivity': 0.9642857142857143, 'Specificity': 0.7164179104477612, 'Precision': 0.4153846153846154, 'AUC-ROC': 0.9501599147121536, 'AUC-PR': 0.8398705952082856, 'Balanced Accuracy': 0.8403518123667377, 'F1 Score': 0.705906996229577}
   Accuracy  Sensitivity  Specificity  Precision  AUC-ROC    AUC-PR  \
0  0.759259     0.964286     0.716418   0.415385  0.95016  0.839871   

   Balanced Accuracy  F1 Score  
0           0.840352  0.705907  
